In [3]:
import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(DefaultAzureCredential(), "SUBSCRIPTION ID", "RESOURCE GROUP", "WORKPLACE NAME")
data = ml_client.data.get(name="foodpanda_reviews_th_en", version="1")

# Load into pandas for inspection
df = pd.read_csv(data.path)
print(df.columns)


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.1.1) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()


Index(['StoreId', 'uuid', 'createdAt', 'updatedAt', 'text', 'isAnonymous',
       'reviewerId', 'replies', 'likeCount', 'isLiked', 'overall',
       'restaurant_food', 'rider'],
      dtype='object')


In [4]:
print(df.columns.tolist())
print(df.dtypes)
print(df.isna().mean().sort_values(ascending=False).head(10))
df.sample(5, random_state=42)

['StoreId', 'uuid', 'createdAt', 'updatedAt', 'text', 'isAnonymous', 'reviewerId', 'replies', 'likeCount', 'isLiked', 'overall', 'restaurant_food', 'rider']
StoreId             object
uuid                object
createdAt           object
updatedAt           object
text                object
isAnonymous           bool
reviewerId          object
replies             object
likeCount          float64
isLiked               bool
overall              int64
restaurant_food      int64
rider              float64
dtype: object
rider          0.518491
StoreId        0.000000
uuid           0.000000
createdAt      0.000000
updatedAt      0.000000
text           0.000000
isAnonymous    0.000000
reviewerId     0.000000
replies        0.000000
likeCount      0.000000
dtype: float64


,StoreId,uuid,createdAt,updatedAt,text,isAnonymous,reviewerId,replies,likeCount,isLiked,overall,restaurant_food,rider
44962,j6sn,dbe20cda-561e-4845-8027-8bd1f74ecf51,2023-08-14T09:13:38Z,2023-08-14T09:13:38Z,กินกี่ครั้งก็อร่อย,False,thzru9wc,[],0.0,False,5,5,5.0
13744,hf3i,4399254b-6bef-4dfb-bef7-7d0b03689668,2023-11-22T13:13:44Z,2023-11-22T13:13:44Z,ผัดกระเพราไม่อร่อยไห้น้อย แต่โรตีอร่อยมาก,False,thall2i7,[],1.0,False,2,2,NaN
14861,a9bl,4920e729-3f41-472b-b669-82ba886e76d4,2023-04-14T14:28:41Z,2023-04-14T14:28:41Z,อร่อย แต่รอนานไปหน่อย,False,thlcb8hl,[],0.0,False,5,5,5.0
35514,g2iw,ae24fd73-73f9-4f67-957f-afde6eceb487,2024-02-14T05:27:16Z,2024-02-14T05:27:16Z,ให้อย่างเยอะเลยอะ อร่อย แต่ติดเค็มนิดนึง รอนาน...,False,th6811ou,[],0.0,False,5,5,5.0
32680,bkgx,a014d1a0-ea6b-4cad-af86-586c122b2c97,2023-10-24T03:28:20Z,2023-10-24T03:28:20Z,บานอฟฟี่อร่อยย,False,thv5qk37,[],0.0,False,5,5,NaN


In [5]:
pd.set_option("display.max_columns", 50)

# Parse ISO timestamps
df["createdAt"] = pd.to_datetime(df["createdAt"], errors="coerce", utc=True)
df["updatedAt"] = pd.to_datetime(df["updatedAt"], errors="coerce", utc=True)

print("Nulls in createdAt/updatedAt:")
print(df[["createdAt","updatedAt"]].isna().mean())

print("\nCreatedAt min/max:")
print(df["createdAt"].min(), " -> ", df["createdAt"].max())

print("\nAny empty text rows?")
print(((df["text"].astype(str).str.strip()=="") | df["text"].isna()).sum())


Nulls in createdAt/updatedAt:
createdAt    0.0
updatedAt    0.0
dtype: float64

CreatedAt min/max:
2023-03-19 22:17:54+00:00  ->  2024-03-20 05:01:04+00:00

Any empty text rows?
0


In [6]:
print("overall value_counts:")
print(df["overall"].value_counts(dropna=False).sort_index())

# Quick sentiment mapping preview (no mutation yet)
def map_sentiment(x):
    if pd.isna(x): return pd.NA
    x = int(x)
    if x in (4,5): return "positive"
    if x == 3:     return "neutral"
    if x in (1,2): return "negative"
    return pd.NA

temp_sent = df["overall"].apply(map_sentiment)
print("\nProposed sentiment distribution:")
print(temp_sent.value_counts(dropna=False))


overall value_counts:
1     9043
2     3776
3     5408
4     5603
5    28655
Name: overall, dtype: int64

Proposed sentiment distribution:
positive    34258
negative    12819
neutral      5408
Name: overall, dtype: int64


In [7]:
# rider has ~51.8% NaN per your output
print("rider NaN rate:", df["rider"].isna().mean())

# Convert likeCount to numeric, cap outliers, show summary
df["likeCount"] = pd.to_numeric(df["likeCount"], errors="coerce")
print("\nlikeCount describe():")
print(df["likeCount"].describe(percentiles=[0.5,0.9,0.99]))

# Optional cap to reduce skew for tabular runs
df["likeCount_capped"] = df["likeCount"].clip(lower=0, upper=df["likeCount"].quantile(0.99))
print("\nCapped likeCount describe():")
print(df["likeCount_capped"].describe())


rider NaN rate: 0.5184909974278366

likeCount describe():
count    52485.000000
mean         0.074174
std          0.299704
min         -1.000000
50%          0.000000
90%          0.000000
99%          1.000000
max          8.000000
Name: likeCount, dtype: float64

Capped likeCount describe():
count    52485.000000
mean         0.066343
std          0.248883
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: likeCount_capped, dtype: float64


In [8]:
# Ensure booleans are 0/1 (some pipelines prefer ints)
df["isAnonymous"] = df["isAnonymous"].astype(int)
df["isLiked"] = df["isLiked"].astype(int)

# replies appears as a JSON-like string "[]"; extract count safely
import ast

def safe_len_list(s):
    if pd.isna(s): return 0
    s = str(s).strip()
    try:
        obj = ast.literal_eval(s)
        return len(obj) if isinstance(obj, list) else 0
    except Exception:
        return 0

df["replies_count"] = df["replies"].apply(safe_len_list)
print("replies_count value_counts (top 5):")
print(df["replies_count"].value_counts().head())


replies_count value_counts (top 5):
0    50547
1     1938
Name: replies_count, dtype: int64


In [9]:
# Create sentiment column
df["sentiment"] = df["overall"].apply(map_sentiment)

# Drop rows without text or sentiment
df = df[~df["text"].isna() & (df["text"].astype(str).str.strip()!="")]
df = df[~df["sentiment"].isna()]

# Date features
df["created_year"]  = df["createdAt"].dt.year
df["created_month"] = df["createdAt"].dt.month
df["created_dow"]   = df["createdAt"].dt.dayofweek
df["created_hour"]  = df["createdAt"].dt.hour

# High-cardinality IDs to drop from modeling (keep a copy if you want lineage)
id_cols = ["uuid","reviewerId","updatedAt"]
keep_ids = df[id_cols].copy()
df_model = df.drop(columns=[c for c in id_cols if c in df.columns])

print("Sentiment distribution after filtering:")
print(df_model["sentiment"].value_counts())

print("\nPreview engineered columns:")
print(df_model[["text","overall","sentiment","isAnonymous","isLiked","likeCount_capped","replies_count","StoreId","created_year","created_month","created_dow","created_hour"]].head(3))


Sentiment distribution after filtering:
positive    34258
negative    12819
neutral      5408
Name: sentiment, dtype: int64

Preview engineered columns:
                                                text  overall sentiment  \
0                         Best food ever. Literally!        5  positive   
1  The meat was far INFERIOR to what I expected.....        3   neutral   
2  อร่อย หมูกรอบสับชิ้นใหญ่ค่ะ ชอบ แถมให้ซอสซีอิ้...        5  positive   

   isAnonymous  isLiked  likeCount_capped  replies_count StoreId  \
0            0        0               0.0              0    zn0n   
1            0        0               0.0              0    geap   
2            0        0               0.0              1    wej9   

   created_year  created_month  created_dow  created_hour  
0          2023             12            4            14  
1          2023              7            4            14  
2          2024              1            1            11  


In [14]:
print("Final dtypes snapshot:")
print(df_model.dtypes)

# Save for AutoML use (parquet is efficient)
out_path = "rjk233"
df_model.to_parquet(out_path, index=False)
print(f"Saved: {out_path}, rows={len(df_model):,}")


Final dtypes snapshot:
StoreId                          object
createdAt           datetime64[ns, UTC]
text                             object
isAnonymous                       int64
replies                          object
likeCount                       float64
isLiked                           int64
overall                           int64
restaurant_food                   int64
rider                           float64
likeCount_capped                float64
replies_count                     int64
sentiment                        object
created_year                      int64
created_month                     int64
created_dow                       int64
created_hour                      int64
dtype: object
Saved: rjk233, rows=52,485


In [15]:
print("Class balance:")
print((df_model["sentiment"].value_counts(normalize=True)*100).round(2).astype(str) + "%")

df_model["text_len"] = df_model["text"].astype(str).str.len()
print("\nText length describe():")
print(df_model["text_len"].describe())

print("\nVery short texts (<5 chars):", (df_model["text_len"]<5).sum())


Class balance:
positive    65.27%
negative    24.42%
neutral      10.3%
Name: sentiment, dtype: object

Text length describe():
count    52485.000000
mean        51.517634
std         46.126781
min          1.000000
25%         18.000000
50%         36.000000
75%         69.000000
max        273.000000
Name: text_len, dtype: float64

Very short texts (<5 chars): 816
